In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import pandas as pd
import numpy as np # linear algebra
import seaborn as sns
import matplotlib.pyplot as plt



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install pandas-alive


In [ ]:
pip install bar_chart_race


In [ ]:
pip install quickda

In [ ]:
# Get Data
data = pd.read_csv('/kaggle/input/reportvehiclesinuseeurope2021fix/ReportVehiclesInUseEurope2021.csv')



In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.mean()

In [ ]:
data.mode()


In [ ]:
#Standard Deviation
std = np.std(data)
print(std)

In [ ]:
#Coefficient of Variation
cv = np.std(data) / np.mean(data)
print(cv)

In [ ]:
#Variance
var_full = np.var(data)
print(var_full)

In [ ]:
import pandas_profiling
report = pandas_profiling.ProfileReport(data)
from IPython.display import display
display(report)

In [ ]:
sns.pairplot(data,palette='bright')
%time

In [ ]:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(data, kind="reg", plot_kws={'line_kws':{'color':'red'}})
plt.show()
%time

In [ ]:
# Importing libraries
from quickda.explore_data import *
from quickda.clean_data import *
from quickda.explore_numeric import *
from quickda.explore_categoric import *
from quickda.explore_numeric_categoric import *
from quickda.explore_time_series import *

In [ ]:
eda_num(data)

In [ ]:
eda_num(data, method="correlation")

In [ ]:
#Add Geocoding
!pip install geopy==1.22.0
import math
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker,Circle
from folium.plugins import MarkerCluster
from geopandas.tools import geocode
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

In [ ]:
point = result.geometry.iloc[0]
print("Latitude:", point.y)
print("Longitude:", point.x)

In [ ]:
def my_geocoder(row):
    try:
        point = geocode(row, provider='nominatim').geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

data[['Latitude', 'Longitude', 'geometry']] = data.apply(lambda x: my_geocoder(x['Country']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(data["Latitude"])) / len(data)) * 100))

# Drop data that were not successfully geocoded
data = data.loc[~np.isnan(data["Latitude"])]
data = gpd.GeoDataFrame(data, geometry=data.geometry)
data.crs = {'init': 'epsg:4326'}
data.head()

# Average Age

In [ ]:
# Create the map
v_map = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2) 

# Add points to the map
mc = MarkerCluster()
for idx, row in data.iterrows(): 
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(Marker(location=[row['Latitude'], row['Longitude']],
                            tooltip=str(row['AverageAge'])+" AverageAge"))
v_map.add_child(mc)

# Display the map
v_map

In [ ]:
# Create a Bubble base map
#Agenda = blue-good , yellow - average , red - bad
m_bubble_focus = folium.Map(location=[39.760982,10.00065], tiles='cartodbpositron', zoom_start=2)

def color_producer(val):
    if val <= 8:
        return 'blue'
    if val >= 8 and val<= 11:
        return 'yellow'
    else:
        return 'red'

# Add a bubble map to the base map
for i in range(0,len(data)):
    Circle(
        location=[data.iloc[i]['Latitude'], data.iloc[i]['Longitude']],
        radius=20,
        color=color_producer(data.iloc[i]['AverageAge'])).add_to(m_bubble_focus)

# Display the map
m_bubble_focus